In [1]:
%load_ext autoreload
%autoreload 1
import torch
import torch.nn as nn
from torch.nn import functional
import torchvision
import numpy as np
import src_loader
from datasets import NaiveResizeGray, ResizeCropGray, BinaryClassification
import datasets
from tqdm.notebook import tqdm
import models

ModuleNotFoundError: No module named 'datasets'

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")
    
TWO_CLASS = True

Running on the GPU


In [3]:
#min_y, max_y, min_x, max_x = datasets.determine_sizes('Data/CheXpert-v1.0', ['Data/CheXpert-v1.0/train-45000-reduced.csv', 'Data/CheXpert-v1.0/valid-5000-reduced.csv'])
#center_crop_size = min(min_y, min_x)
#print(min_y, max_y, min_x, max_x)

In [4]:
training_set = BinaryClassification('Data/CheXpert-v1.0/train-45000-reduced.csv')#, mean = 0.00010563735961914062, std=0.010277460797675398)
#print(training_set.mean, training_set.std)
validation_set = BinaryClassification('Data/CheXpert-v1.0/valid-5000-reduced.csv')

In [5]:
BATCH_SIZE = 32
NUM_OF_WORKERS = 20

VAL_BATCH_SIZE = 16
VAL_NUM_WORKERS = 10

train_loader = torch.utils.data.DataLoader(training_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_OF_WORKERS, pin_memory=True)
val_loader = torch.utils.data.DataLoader(validation_set, batch_size=VAL_BATCH_SIZE, shuffle=True, num_workers=VAL_NUM_WORKERS, pin_memory=True)

In [6]:
%aimport models
%aimport datasets
#%env CUDA_LAUNCH_BLOCKING=1
#model = models.SimpleNet(training_set.num_pathologies, two_class=TWO_CLASS).to(device)
model = models.testNet_2class(training_set.num_pathologies, training_set.output_size).to(device)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_per_feature = nn.CrossEntropyLoss()
criterion = nn.BCEWithLogitsLoss()

testNet_2class(
  (conv1): Conv2d(1, 4, kernel_size=(12, 12), stride=(6, 6))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1600, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=5, bias=True)
)


In [7]:
def loss_3class(y_hat, y):
    
    batch_loss = loss_per_feature(y_hat[:,0:2], y[:,0])
    for i in range(int((y_hat.shape[1]-2)/3)):
        batch_loss += loss_per_feature(y_hat[:,(i*3+2):((i+1)*3+2)], y[:,i+1])
    return batch_loss

In [8]:
val_iter = iter(val_loader)

def validate_batch():
    with torch.no_grad():
        global val_iter
        try:
            val_x, val_y = next(val_iter)
        except StopIteration:
            val_iter = iter(val_loader)
            val_x, val_y = next(val_iter)
            
        val_x, val_y = val_x.to(device), val_y.to(device)
        y_hat = model(val_x)
        
        return loss(y_hat, val_y)
    
def validate_batch_2class():
    with torch.no_grad():
        global val_iter
        try:
            val_x, val_y = next(val_iter)
        except StopIteration:
            val_iter = iter(val_loader)
            val_x, val_y = next(val_iter)
            
        val_x, val_y = val_x.to(device), val_y.to(device)
        y_hat = model(val_x)
        
        return criterion(y_hat, val_y)

In [9]:
def total_validation_loss_3class():
    with torch.set_grad_enabled(False):
        val_loss = 0
        for val_x, val_y in val_loader:
            x, y = val_x.to(device), val_y.to(device)
            
            val_loss += loss(model(x), y)
        empirical_loss = val_loss.item()/validation_set.__len__()
        print('Empirical Validation Loss: {}'.format(empirical_loss))
    return empirical_loss

def total_validation_loss_2class():
    with torch.set_grad_enabled(False):
        val_loss = 0
        for val_x, val_y in val_loader:
            x, y = val_x.to(device), val_y.to(device)
            
            val_loss += criterion(model(x), y)
        empirical_loss = val_loss.item()/validation_set.__len__()
        print('Empirical Validation Loss: {}'.format(empirical_loss))
    return empirical_loss

In [10]:
EPOCHS = 3

print(device)

total_validation_loss_2class()

for epoch in range(EPOCHS):
    with tqdm(train_loader) as pbar:
        for i, batch in enumerate(pbar):
            
            x, y = batch
            #print(x[0])
            #break
            
            if i % 10 == 0:
                val_loss = validate_batch_2class().item()/VAL_BATCH_SIZE
            
            
            x = x.to(device)
            y = y.to(device)
            
            y_hat = model(x)
            #print(y.dtype)
            #print(y_hat.dtype)
            batch_loss = criterion(y_hat, y)
            
            #print(y_hat, y)
            
            pbar.set_postfix(loss=batch_loss.item()/BATCH_SIZE, val_loss=val_loss)
            
            
            batch_loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
    total_validation_loss_2class()

cuda:0
Empirical Validation Loss: 0.04225258255004883



Empirical Validation Loss: 0.03290546417236328



Empirical Validation Loss: 0.03351863861083984



Empirical Validation Loss: 0.032892040252685543


In [11]:
torch.save(model, 'trained/NaiveResizeGray/first_reference.pt')